# EDA — Workshop ETL

Análisis Exploratorio de Datos (EDA)
Objetivo
Analizar la calidad, estructura y características de los datasets crudos (customers, products, transactions) para informar el diseño del proceso ETL.

1. Dataset: customers.csv
Columnas: customer_id, country, signup_ts, marketing_source

Análisis de Calidad:

customer_id: Parece ser un identificador único y consistente.

country: Problema Detectado. Existen valores nulos/vacíos. Estos deberán ser tratados (ej. imputar con "Desconocido") para mantener la integridad de los registros.

signup_ts: Fechas en formato YYYY-MM-DD. Deberán ser convertidas a tipo datetime para análisis temporal.

marketing_source: Categórico. Parece consistente.

2. Dataset: products.csv
Columnas: product_id, category, price_list

Análisis de Calidad:

Este dataset parece estar limpio y listo para usarse como una tabla de dimensión.

product_id: Identificador único.

category: Categórico.

price_list: Numérico. Deberá ser convertido a tipo float.

3. Dataset: transactions.csv (Fact Table)
Este es el dataset más crítico y con más problemas.

Columnas: txn_id, customer_id, product_id, amount, currency, ts, status

Análisis de Calidad:

txn_id: Identificador único de la transacción.

customer_id: Problema Detectado. Existen valores nulos/vacíos. Las transacciones sin cliente no pueden ser atribuidas, por lo que estas filas deberán ser eliminadas.

product_id: Problema Detectado. Existen valores nulos/vacíos. Similar al customer_id, estas filas deberán ser eliminadas.

amount: Problema Crítico Detectado. Los datos están "sucios".

Contienen símbolos de moneda (ej. $, €).

Usan comas como separador decimal (ej. "25,16").

La columna debe ser limpiada (eliminar símbolos, reemplazar , por .) y convertida a float.

currency: Categórico (USD, EUR, MXN, BRL). Se usará para la conversión de moneda.

ts: Problema Detectado. Contiene timestamps completos (ej. 2025-08-03 11:46:00), pero también valores incompletos (ej. t0000012 solo tiene fecha). Las filas sin un timestamp completo deberán ser eliminadas ya que no se pueden usar para análisis de series temporales.

status: Problema Crítico Detectado. Los datos son inconsistentes.

Múltiples valores para el mismo estado: paid, Paid, PAID.

Múltiples valores para reembolso: refunded, Refunded.

La columna debe ser normalizada a un conjunto de estados estándar (ej. paid, refunded, cancelled).

4. Dataset: exchange_rates.csv (Por crear)
Columnas: currency, rate_to_usd

Datos (Según README):

USD: 1.0

EUR: 1.08

MXN: 0.058

BRL: 0.18

Acción ETL: Este archivo será creado por el script de extracción y luego leído para ser usado en la transformación.

Conclusión del EDA
El proceso ETL debe centrarse en:

Limpiar transactions: Manejar nulos (drop), normalizar status y limpiar/convertir amount.

Limpiar customers: Imputar nulos en country.

Convertir Moneda: Usar exchange_rates.csv para crear una nueva columna amount_usd en la tabla de transacciones.

Enriquecer: Unir las transacciones limpias con los datos de clientes y productos.

Cargar: Almacenar el resultado final en un formato eficiente (Parquet).

In [ ]:

import pandas as pd
from pathlib import Path

RAW = Path("../data/raw")
tx = pd.read_csv(RAW/"transactions.csv")
cu = pd.read_csv(RAW/"customers.csv")
pr = pd.read_csv(RAW/"products.csv")
fx = pd.read_csv(RAW/"exchange_rates.csv")
cr = pd.read_csv(RAW/"country_region.csv")
tx.head()


## TODO: perfilado, monedas, clientes, productos, temporal